# Clustering Crypto

In [39]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [73]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df.set_index("Unnamed: 0", inplace=True)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [74]:
# Keep all the cryptocurrencies that are being traded.
crypto1_df=crypto_df[crypto_df.IsTrading == True]
crypto1_df['IsTrading'].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [75]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto1_df['Algorithm'].describe()

count       1144
unique        89
top       Scrypt
freq         394
Name: Algorithm, dtype: object

In [76]:
# Remove the "IsTrading" column. 
crypto2_df = crypto1_df.drop(columns=['IsTrading'])
crypto2_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [77]:
# Remove rows that have at least 1 null value.
crypto3_df=crypto2_df.dropna()
crypto3_df['Algorithm'].describe()

count        685
unique        80
top       Scrypt
freq         227
Name: Algorithm, dtype: object

In [78]:
# Keep the rows where coins are mined.
crypto4_df=crypto3_df[crypto3_df.TotalCoinsMined > 0]
crypto4_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [79]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df=crypto4_df.loc[:,'CoinName'].reset_index()
crypto_names_df.head()

,Unnamed: 0,CoinName
0,42,42 Coin
1,404,404Coin
2,1337,EliteCoin
3,BTC,Bitcoin
4,ETH,Ethereum


In [80]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto5_df = crypto4_df.drop(columns = ['CoinName'])
crypto5_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [81]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto5_df, columns=["Algorithm", "ProofType"])
X.head(100)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XJO,3.919574e+07,45000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XST,3.309373e+07,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ZCC,1.479902e+08,1000000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [82]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
len(X_scaled)

532

### Deliverable 2: Reducing Data Dimensions Using PCA

In [83]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

In [84]:
# Create a DataFrame with the three principal components.
pcs_df=pd.DataFrame(data = X_pca, columns=["PC 1","PC 2","PC 3"])
pcs_df=pd.concat([crypto_names_df,pcs_df],axis=1).drop(columns=['CoinName']).set_index('Unnamed: 0')
pcs_df.index.name = None
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.330281,1.058408,-0.582522
404,-0.313608,1.058651,-0.582992
1337,2.304409,1.597746,-0.756277
BTC,-0.145018,-1.349552,0.176212
ETH,-0.154734,-2.035386,0.407028


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [85]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)
elbow_data = {"k" : k, "inertia" : inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [86]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

predictions = model.predict(pcs_df)


In [87]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df=pd.concat([crypto5_df,pcs_df],axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
crypto_names2_df=crypto_names_df.set_index('Unnamed: 0')
clustered_df=pd.concat([clustered_df,crypto_names2_df],axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_
clustered_df.index.name = None

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.330281,1.058408,-0.582522,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.313608,1.058651,-0.582992,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.304409,1.597746,-0.756277,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145018,-1.349552,0.176212,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.154734,-2.035386,0.407028,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160589,-1.064369,0.004493,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.400381,1.100364,-0.473608,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.158214,-2.162954,0.413198,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.153175,-2.035474,0.406998,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.152392,-2.091506,0.514663,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [88]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df, x="PC 1", y="PC 2", z="PC 3",
    color="Class", symbol="Class", width=800,
    hover_name="CoinName", hover_data="")
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [89]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [90]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df['CoinName'].count()} tradable cryptocurrencies")

There are 532 tradable cryptocurrencies


In [91]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_new=MinMaxScaler().fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
X_new

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [92]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df=pd.DataFrame(data=X_new, columns=['TotalCoinSupply','TotalCoinsMined'])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df=pd.concat([plot_df,crypto_names_df],axis=1).set_index('Unnamed: 0')
plot_df.index.name = None

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame.
plot_df['Class']=clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [93]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot(x='TotalCoinsMined',y='TotalCoinSupply', by='Class', kind='scatter', hover_cols='CoinName')


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)